# 1. 设计`ChangEDataBase`

由于嫦娥公开数据文件数量众多，文件名超长，所以这里专门编写了一个类，将数据的远程查询、下载、缓存、及本地标签查询等功能集成在一起，便于管理和访问。

In [15]:
import os
import re
import requests
import time
import atool

class ChangEDataBase:
    def __init__(self, path='./data'):
        self.__cache_path = path
        self.__data = []
        self.info()
        pass
    
    def get_ChangE_download_link(pageNo, pageSize, task_id='CE4', data_type='SCI', payload='PCAM', data_grade='2B', date_start='default', date_end='today'):
        """
        query_remote
        根据定制参数，查询远程服务器
        """
        
        # 默认情况下，根据任务设置查询的起始时间
        if date_start == 'default':
            if task_id == 'CE5':
                date_start = '2020-12-01 00:00:00'
            if task_id == 'CE4':
                date_start = '2019-01-04 00:00:00'
            if task_id == 'CE3':
                date_start = '2009-12-31 00:00:00'
            if task_id == 'CE2':
                date_start = '2010-10-01 00:00:00'
            if task_id == 'CE1':
                date_start = '2007-10-24 00:00:00'

        # 默认情况下，设置查询的结束时间为当天
        if date_end == 'today':
            date_end = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

        # 组装参数
        url = 'https://' + 'moon.bao.ac.cn' + '/proSearch!proSearchList.search'
        inputJsonString = '{"globalLang":null,"DM_DTYPE":"%s","DM_TASKID":"%s","DM_ZHNAME":"%s","DM_GRADE":"%s","DM_CBTIMEStart":"%s","DM_CBTIMEend":"%s","order_by":"desc","qsList":"","qsStart":"","qsEnd":"","bianhao":""}' % (data_type, task_id, payload, data_grade, date_start, date_end)
        param = {
            '_dc': '1615623500055', 
            'pageNo': str(pageNo), 
            'pageSize': str(pageSize), 
            'inputJsonString': inputJsonString, 
            'page': str(pageNo), 
            'start': str((pageNo-1)*pageSize), 
            'limit': str(pageSize)}
        
        # 访问网站，查询
        r = requests.get(url, param)
        if r.ok:
            link = []
            link.append(r.json()['totalCount'])
            for data in r.json()['returndata']:
                #print(atool.pretty(data))
                # 真实的下载链接
                link.append(data['dmStorepath'].replace('/PUBDATA/', 'http://moon.bao.ac.cn/cedownload/'))

            return link
        else:
            return None
    
    def query_remote(task_id, payload, data_type, data_grade, date_start, date_end):
        """
        get_ChangE_download_link()函数的套壳，不需要指定页码和页面大小
        """
        
        # 先查第一页，目的是获得数据总条目
        l = get_ChangE_download_link(1, 10, task_id, data_type, payload, data_grade, date_start, date_end)
        totalcount = l[0]
        print(totalcount)
        
    
    def scan_cache(self, with_hidden=False):
        self.__data = []
        for f in os.listdir(self.__cache_path):
            if with_hidden == False and f[0] == '.':
                pass
            else:
                self.__data.append(f)
        count = len(self.__data)
        return count
    
    def info(self):
        print('嫦娥数据库 v0.1')
        print('缓存目录 : %s' % (self.__cache_path))
        print('缓存数目 ：%d' % (self.scan_cache()))
        #for f in os.listdir(self.__cache):
        #    print(f)
        
    def query(self, qtask='CE4', qseq='*', qbatch='*', qcolor='*', qsite='*', qpayload='*', qdata_type='*', qdata_grade='*'):
        """
        发起一次本地查询，遍历缓存目录中的所有文件，并返回符合条件的文件名
        """
        def check_tag(filename, tags):
            # 数据文件名形如：
            # CE4_GRAS_PCAML-C-000_SCI_N_20200227111928_20200227111928_0121_B.2BL_2.png
            match_obj = re.match( r'(.*)_(.*)_(.*)-(.*)-(.*)_(.*)_(.*)_([0-9]*)_([0-9]*)_([0-9]{4})_(.*)\.(.*)', filename)
            # 提取出文件名中的各种标签
            if match_obj != None:
                task = match_obj.group(1) # 任务
                site = match_obj.group(2) # 站点
                payload = match_obj.group(3) # 载荷，如果是PCAM的话，还需要指出是左眼还是右眼
                color = match_obj.group(4) # C、Q
                seq = match_obj.group(5) # 序号
                data_type = match_obj.group(6) # 数据类型
                unknown = match_obj.group(7) # 未知
                start_time = match_obj.group(8) # 起时间
                end_time = match_obj.group(9) # 止时间
                batch = match_obj.group(10) # 序号？
                data_grade = match_obj.group(11) # 数据等级
                ext = match_obj.group(12) # 扩展名
            else:
                return None
                
            ftags = [task, seq, batch, color, site, payload, data_type, data_grade]
                
            # 逐个检查标签是否匹配
            checklist = []
            for i in range(len(tags)):
                t = tags[i]
                f = ftags[i]
                if t == '*':
                    checklist.append(True)
                elif t == f:
                    checklist.append(True)
                else:
                    checklist.append(False)
            return checklist
        
        result = []

        # 依次判断文件名是否满足查询
        for f in self.__data:
            checked = True
            checklist = check_tag(f, [qtask, qseq, qbatch, qcolor, qsite, qpayload, qdata_type, qdata_grade])
            if checklist == None:
                print('注意：文件名格式无法匹配 %s' % (f))
            else:
                for c in checklist:
                    if c == False:
                        checked = False
                        break
                if checked == True:
                    result.append(f)
        
        return result   

In [39]:
db = ChangEDataBase(path='./data')
q = db.query(qtask='CE4', qpayload='PCAML', qbatch='0102')
print('查询结果数目：%d\n' % (len(q)))
print(atool.pretty(sorted(q)))

嫦娥数据库 v0.1
缓存目录 : ./data
缓存数目 ：2100
查询结果数目：112

[
    CE4_GRAS_PCAML-Q-000_SCI_N_20191223064003_20191223064003_0102_B.2B
    CE4_GRAS_PCAML-Q-000_SCI_N_20191223064003_20191223064003_0102_B.2BL
    CE4_GRAS_PCAML-Q-001_SCI_N_20191223064123_20191223064123_0102_B.2B
    CE4_GRAS_PCAML-Q-001_SCI_N_20191223064123_20191223064123_0102_B.2BL
    CE4_GRAS_PCAML-Q-002_SCI_N_20191223064243_20191223064243_0102_B.2B
    CE4_GRAS_PCAML-Q-002_SCI_N_20191223064243_20191223064243_0102_B.2BL
    CE4_GRAS_PCAML-Q-003_SCI_N_20191223064403_20191223064403_0102_B.2B
    CE4_GRAS_PCAML-Q-003_SCI_N_20191223064403_20191223064403_0102_B.2BL
    CE4_GRAS_PCAML-Q-004_SCI_N_20191223064523_20191223064523_0102_B.2B
    CE4_GRAS_PCAML-Q-004_SCI_N_20191223064523_20191223064523_0102_B.2BL
    CE4_GRAS_PCAML-Q-005_SCI_N_20191223064643_20191223064643_0102_B.2B
    CE4_GRAS_PCAML-Q-005_SCI_N_20191223064643_20191223064643_0102_B.2BL
    CE4_GRAS_PCAML-Q-006_SCI_N_20191223064803_20191223064803_0102_B.2B
    CE4_GRAS_PCAML-Q-